In [129]:
import pandas as pd
import string
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
import pickle
import preprocess
import datetime

In [7]:
training_data = pd.read_csv('trainingandtestdata/training.1600000.processed.noemoticon.csv', header=None, names=['Sentiment', 'ID', 'date', 'query', 'user', 'tweet'])

In [5]:
test_data = pd.read_csv('trainingandtestdata/testdata.manual.2009.06.14.csv', header=None, names=['Sentiment', 'ID', 'date', 'query', 'user', 'tweet'])

In [67]:
print len(training_data)
print len(test_data)

1600000
498


In [51]:
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['RT', 'rt', 'via', 'http', 'https', ':/']

def word_feats(words):
    return dict([(word, True) for word in words.split(' ') if word not in stop])

training_data['feats'] = training_data['tweet'].apply(word_feats)
test_data['feats'] = test_data['tweet'].apply(word_feats)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:5: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [52]:
training_data.head()

,Sentiment,ID,date,query,user,tweet,feats
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","{u'': True, u'it.': True, u';D': True, u'Third..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,"{u'': True, u'School': True, u'texting': True,..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,"{u'': True, u'50%': True, u'Managed': True, u'..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"{u'body': True, u'': True, u'like': True, u'fi..."
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","{u'no,': True, u'behaving': True, u'there.': T..."


In [53]:
test_data.head()

,Sentiment,ID,date,query,user,tweet,feats
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...,"{u'fantastic': True, u'I': True, u'Kindle2.': ..."
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...,"{u'': True, u'childs': True, u'Love': True, u'..."
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck...","{u'assesment': True, u'fucking': True, u'rocks..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...,"{u'back.': True, u'months': True, u'love': Tru..."
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...,"{u'': True, u'perfect': True, u'@mikefish': Tr..."


In [54]:
training_data.to_pickle('pkl/nb_train.pkl')
test_data.to_pickle('pkl/nb_test.pkl')

In [58]:
def transform_feat(df):
    
    feats = []
    
    for r in df.iterrows():
        sent = r[1][0]
        feat = r[1][6]
        if sent == 0:
            feats.append((feat, 'neg'))
        elif sent == 4:
            feats.append((feat, 'pos'))
        
        
    return feats

In [59]:
trainfeats = transform_feat(training_data)
testfeats = transform_feat(test_data)

In [64]:
with open("pkl/trainfeats.pkl", "wb") as f:
    pickle.dump(trainfeats, f)
    
with open("pkl/testfeats.pkl", "wb") as f:
    pickle.dump(testfeats, f)

In [66]:
classifier = NaiveBayesClassifier.train(trainfeats)
print 'accuracy:', nltk.classify.util.accuracy(classifier, testfeats)
classifier.show_most_informative_features()

accuracy: 0.810584958217
Most Informative Features
                   Died! = True              neg : pos    =    140.3 : 1.0
                 me..its = True              pos : neg    =    114.3 : 1.0
                 bummed. = True              neg : pos    =     97.0 : 1.0
                 McMahon = True              neg : pos    =     76.3 : 1.0
                     447 = True              neg : pos    =     74.3 : 1.0
                  Farrah = True              neg : pos    =     72.3 : 1.0
             @Banksyart2 = True              pos : neg    =     61.7 : 1.0
                   Boooo = True              neg : pos    =     53.0 : 1.0
                 *cries* = True              neg : pos    =     51.0 : 1.0
                    sadd = True              neg : pos    =     49.0 : 1.0


In [114]:
classifier.classify(testfeats[0])

AttributeError: 'tuple' object has no attribute 'copy'

In [69]:
classifier.labels()

['neg', 'pos']

In [70]:
with open('pkl/NB.pkl', 'w') as f:
    pickle.dump(classifier, f)

In [71]:
df = pd.read_pickle('pkl/brexit.pkl')

In [127]:
df.head(1).T

,0
accessLevel,0
contributors,[]
createdAt,1459727999000
currentUserRetweetId,-1
extendedMediaEntities,[]
favoriteCount,0
geoLocation,None
hashtagEntities,"[{u'start': 37, u'end': 43, u'text': u'EUref'}..."
id,716777296229765120
inReplyToScreenName,None


In [161]:
df.sort_values(by='retweetCount', ascending=False).head(1).transpose()

,1614
accessLevel,0
contributors,[]
createdAt,1459718702000
currentUserRetweetId,-1
extendedMediaEntities,[]
favoriteCount,0
geoLocation,None
hashtagEntities,"[{u'start': 139, u'end': 140, u'text': u'Brexi..."
id,716738302540431364
inReplyToScreenName,None


In [164]:
print df.sort_values(by='retweetCount', ascending=False).head(1).values

[[0 [] 1459718702000 -1 [] 0 None
  [{u'start': 139, u'end': 140, u'text': u'Brexit'}] 716738302540431364
  None -1 -1 False False False False u'en' [] None None -1 None True 1681
  False
  {u'mediaEntities': [], u'retweetCount': 1681, u'contributors': [], u'text': u"Brussels, de facto capital of the EU, is also the jihadist capital of Europe. And the Remainers dare to say we're safer in the EU! #Brexit", u'inReplyToUserId': -1, u'retweet': False, u'withheldInCountries': None, u'quotedStatusId': -1, u'isTruncated': False, u'createdAt': 1458632736000, u'urlentities': [], u'userMentionEntities': [], u'hashtagEntities': [{u'start': 131, u'end': 138, u'text': u'Brexit'}], u'source': u'<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', u'quotedStatus': None, u'retweetedStatus': None, u'id': 712183431355047937, u'extendedMediaEntities': [], u'favoriteCount': 1569, u'inReplyToScreenName': None, u'scopes': None, u'rateLimitStatus': None, u'retweetedByMe': False, u'user': {u'p

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127143 entries, 0 to 6998
Data columns (total 38 columns):
accessLevel              127143 non-null int64
contributors             127143 non-null object
createdAt                127143 non-null int64
currentUserRetweetId     127143 non-null int64
extendedMediaEntities    127143 non-null object
favoriteCount            127143 non-null int64
geoLocation              45 non-null object
hashtagEntities          127143 non-null object
id                       127143 non-null int64
inReplyToScreenName      9873 non-null object
inReplyToStatusId        127143 non-null int64
inReplyToUserId          127143 non-null int64
isFavorited              127143 non-null bool
isPossiblySensitive      127143 non-null bool
isRetweeted              127143 non-null bool
isTruncated              127143 non-null bool
lang                     127143 non-null object
mediaEntities            127143 non-null object
place                    2162 non-null object
qu

In [109]:
hashtag_stay = ['yes2eu', 'yestoeu', 'betteroffin', 'votein', 'ukineu', 'bremain', 'strongerin', 'leadnotleave', 'voteremain', 'remain', 'stay']
hashtag_leave = ['no2eu', 'notoeu', 'betteroffout', 'voteout', 'britainout', 'leaveeu', 'loveeuropeleaveeu', 'voteleave', 'beleave', 'leave', 'out']
vocab_stay = ['stay', 'remain', 'in']
vocab_leave = ['leave', 'exit', 'out']

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['RT', 'rt', 'via', 'http', 'https', ':/']

def rule_based_tweets(df):
    
    subject = []
    
    for i, d in df.iterrows():
        stay = False
        leave = False
        for h in d.hashtagEntities:
            if not (stay and leave):
#                 print h['text']
                if h['text'].lower() in hashtag_stay:
                    stay = True
                if h['text'].lower() in hashtag_leave:
                    leave = True
#         print d

#         print "hash debug", "stay", stay, "leave", leave

#         print type(d.text), d.text
        if not (stay and leave):
            terms = [term for term in preprocess.preprocess(d.text)]
            no_stop = [term for term in terms if term not in stop]
            words = [term for term in no_stop if not term.startswith(('#', '@'))]
#             print words
            for word in words:
                if not stay and leave:
                    if word.lower() in vocab_stay:
                        stay = True
                    if h['text'].lower() in vocab_leave:
                        leave = True
                        

        
        if stay and not leave:
            subject.append('stay')
        elif not stay and leave:
            subject.append('leave')
        else:
            subject.append('neutral')
#         print "text debug", "stay", stay, "leave", leave
            
    return subject
        
subjects_strict = rule_based_tweets(df)

In [106]:
df['subject'] = subjects

In [108]:
df['subject'].value_counts()

leave      104753
neutral     14171
stay         8219
Name: subject, dtype: int64

In [110]:
df['subject_strict'] = subjects_strict

In [111]:
df['subject_strict'].value_counts()

neutral    82584
leave      31904
stay       12655
Name: subject_strict, dtype: int64

In [113]:
df['feats'] = df['text'].apply(word_feats)

In [119]:
df['sentiment'] = classifier.classify_many(df['feats'].values)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/nltk/classify/naivebayes.py:97: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if (label, fname) in self._feature_probdist:


In [120]:
df['sentiment'].value_counts()

neg    66771
pos    60372
Name: sentiment, dtype: int64

In [123]:
for g, d in df.groupby('subject_strict'):
    print g
    print d['sentiment'].value_counts()

leave
pos    16041
neg    15863
Name: sentiment, dtype: int64
neutral
neg    44442
pos    38142
Name: sentiment, dtype: int64
stay
neg    6466
pos    6189
Name: sentiment, dtype: int64


In [149]:
for g, d in df.groupby('subject'):
    print g
    val = d['sentiment'].value_counts()
    print val

leave
neg    54498
pos    50255
Name: sentiment, dtype: int64
neutral
neg    8474
pos    5697
Name: sentiment, dtype: int64
stay
pos    4420
neg    3799
Name: sentiment, dtype: int64


In [152]:
type(val)

pandas.core.series.Series

In [158]:
val['pos']

4420

In [126]:
df.to_pickle('pkl/brexit.pkl')

In [146]:
df['date'] = df['createdAt'].apply(lambda x: datetime.fromtimestamp(x/1e3))

In [147]:
df['date'].value_counts()

2016-04-07    26124
2016-04-06    22095
2016-04-08    18902
2016-04-05    18339
2016-04-04    16771
2016-04-09     7061
2016-04-12     7000
2016-04-03     5201
2016-04-11     3809
2016-04-10     1360
2016-04-13      481
Name: date, dtype: int64

In [159]:
for g, d in df.groupby('date'):    
    print g
    for g_, d_ in d.groupby('subject_strict'):
        print g_
        print d_['sentiment'].value_counts()

2016-04-03
leave
neg    1006
pos     961
Name: sentiment, dtype: int64
neutral
neg    1510
pos    1341
Name: sentiment, dtype: int64
stay
neg    199
pos    184
Name: sentiment, dtype: int64
2016-04-04
leave
neg    2013
pos    1948
Name: sentiment, dtype: int64
neutral
neg    5826
pos    5263
Name: sentiment, dtype: int64
stay
neg    1023
pos     698
Name: sentiment, dtype: int64
2016-04-05
leave
neg    2090
pos    1937
Name: sentiment, dtype: int64
neutral
neg    6568
pos    6016
Name: sentiment, dtype: int64
stay
pos    973
neg    755
Name: sentiment, dtype: int64
2016-04-06
leave
neg    3318
pos    2383
Name: sentiment, dtype: int64
neutral
neg    8022
pos    6432
Name: sentiment, dtype: int64
stay
neg    1084
pos     856
Name: sentiment, dtype: int64
2016-04-07
leave
pos    3387
neg    3051
Name: sentiment, dtype: int64
neutral
neg    9167
pos    8175
Name: sentiment, dtype: int64
stay
pos    1390
neg     954
Name: sentiment, dtype: int64
2016-04-08
leave
pos    2905
neg    2030
Nam